In [1]:
import pymysql
import requests


下載分析pm2.5 opendata資料

In [2]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [3]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\105812397-DJANGO\MYSQL\PM2.5-MYSQL\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [6]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '1',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '2',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '13',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate':

In [7]:
len(datas)

1000

In [8]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '4',
 'datacreationdate': '2025-08-20 19:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [3]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [2]:
conn=pymysql.connect(
host="mysql-4b43e3-harry99072222-e5a9.i.aivencloud.com",
user="avnadmin",
password="AVNS_M2BtU4CFe_Uot0Hf6qI",
port=17191,
database="defaultdb"
)

cursor=conn.cursor()
cursor

In [4]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [18]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}','{}','{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣','4','2025-08-20 19:00','μg/m3')"

In [19]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入
- 加上 ignore

In [31]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [33]:
conn.close()

In [32]:
import pandas as pd

In [35]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas

((1, '員林', '彰化縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (2, '大城', '彰化縣', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (3, '富貴角', '新北市', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4, '麥寮', '雲林縣', 1, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (5, '關山', '臺東縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (6, '馬公', '澎湖縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (7, '馬祖', '連江縣', 13, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (8, '埔里', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (9, '復興', '高雄市', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (10, '永和', '新北市', 8, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (11, '竹山', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (12, '中壢', '桃園市', 10, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (13, '三重', '新北市', 11, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (14, '冬山', '宜蘭縣', 9, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (15, '宜蘭',

In [37]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,1,員林,彰化縣,4,2025-08-20 19:00:00,μg/m3
1,2,大城,彰化縣,5,2025-08-20 19:00:00,μg/m3
2,3,富貴角,新北市,5,2025-08-20 19:00:00,μg/m3
3,4,麥寮,雲林縣,1,2025-08-20 19:00:00,μg/m3
4,5,關山,臺東縣,4,2025-08-20 19:00:00,μg/m3
...,...,...,...,...,...,...
476,477,新莊,新北市,12,2025-08-20 13:00:00,μg/m3
477,478,板橋,新北市,10,2025-08-20 13:00:00,μg/m3
478,479,土城,新北市,12,2025-08-20 13:00:00,μg/m3
479,480,汐止,新北市,7,2025-08-20 13:00:00,μg/m3


In [41]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-20 19:00:00
1,大城,2025-08-20 19:00:00
2,富貴角,2025-08-20 19:00:00
3,麥寮,2025-08-20 19:00:00
4,關山,2025-08-20 19:00:00
...,...,...
476,新莊,2025-08-20 13:00:00
477,板橋,2025-08-20 13:00:00
478,土城,2025-08-20 13:00:00
479,汐止,2025-08-20 13:00:00


### 取得各縣市站點的平均值

In [4]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

7.2848

In [7]:
sqlstr="""
select county,round(avg(pm25),2) from pm25 group by county;
"""

cursor.execute(sqlstr)
cursor.fetchall()

(('彰化縣', Decimal('8.76')),
 ('新北市', Decimal('4.70')),
 ('雲林縣', Decimal('8.40')),
 ('臺東縣', Decimal('2.52')),
 ('澎湖縣', Decimal('5.50')),
 ('金門縣', Decimal('8.38')),
 ('南投縣', Decimal('9.16')),
 ('高雄市', Decimal('10.59')),
 ('桃園市', Decimal('8.11')),
 ('宜蘭縣', Decimal('3.04')),
 ('臺北市', Decimal('4.62')),
 ('花蓮縣', Decimal('2.85')),
 ('屏東縣', Decimal('7.38')),
 ('臺南市', Decimal('8.53')),
 ('嘉義市', Decimal('7.23')),
 ('嘉義縣', Decimal('9.33')),
 ('臺中市', Decimal('9.24')),
 ('苗栗縣', Decimal('8.69')),
 ('新竹市', Decimal('8.23')),
 ('新竹縣', Decimal('7.12')),
 ('基隆市', Decimal('3.25')),
 ('連江縣', Decimal('6.50')))

In [8]:
conn.close()